In [1]:
from pathlib import Path
import subprocess

REPO_URL = "https://github.com/seoultechpse/fenicsx-colab.git"
ROOT = Path("/content")
REPO_DIR = ROOT / "fenicsx-colab"

subprocess.run(["git", "clone", REPO_URL, str(REPO_DIR)], check=True)

USE_COMPLEX = False  # <--- Set True ONLY if you need complex PETSc
USE_CLEAN = False    # <--- Set True to remove existing environment

opts_str = " ".join(
  [o for c, o in [(USE_COMPLEX, "--complex"), (USE_CLEAN, "--clean")] if c]
)

get_ipython().run_line_magic("run", f"{REPO_DIR / 'setup_fenicsx.py'} {opts_str}")

🔧 FEniCSx Setup Configuration
PETSc type      : real
Clean install   : False

⚠️  Google Drive not mounted — using local cache (/content)

🔧 Installing FEniCSx environment...

🔍 Verifying PETSc type...
✅ Installed: Real PETSc (float64)

✨ Loading FEniCSx Jupyter magic... %%fenicsx registered

✅ FEniCSx setup complete!

Next steps:
  1. Run %%fenicsx --info to verify installation
  2. Use %%fenicsx in cells to run FEniCSx code
  3. Use -np N for parallel execution (e.g., %%fenicsx -np 4)

📌 Note: Real PETSc is installed
   - Recommended for most FEM problems
   - For complex problems, reinstall with --complex


---

In [2]:
%%fenicsx -np 4

import numpy as np
from dolfinx import mesh, fem
from dolfinx.fem import petsc
from dolfinx.io import XDMFFile
import ufl
from mpi4py import MPI
from petsc4py import PETSc

# =============================================================================
# PROBLEM SETUP
# =============================================================================

# Create mesh
domain = mesh.create_unit_square(
    MPI.COMM_WORLD, 100, 100,
    cell_type=mesh.CellType.quadrilateral
)

# Rank for controlling print statements
rank = domain.comm.rank

# Define diffusion coefficient (DG-0 space)
Q = fem.functionspace(domain, ("DG", 0))
c = fem.Function(Q)
c.x.array[:] = 0.1

# Set different diffusivity in left region
cells_left = mesh.locate_entities(
    domain, domain.topology.dim,
    lambda x: x[0] <= 0.7 + 1e-12
)
c.x.array[cells_left] = 0.01
c.x.scatter_forward()

# =============================================================================
# FUNCTION SPACE AND INITIAL CONDITION
# =============================================================================

# Define function space for temperature
V = fem.functionspace(domain, ("Lagrange", 1))
T = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

# Initial condition
T_n = fem.Function(V)
T_n.interpolate(lambda x: np.sin(np.pi * x[0]) * np.sin(np.pi * x[1]))

# =============================================================================
# TIME STEPPING PARAMETERS
# =============================================================================

dt = fem.Constant(domain, 0.01)
t = 0.0
T_end = 2.0

# Source term and boundary value
f = fem.Function(V)
f.x.array[:] = 0
g = fem.Function(V)
g.x.array[:] = 0

# =============================================================================
# BOUNDARY CONDITIONS
# =============================================================================

domain.topology.create_connectivity(domain.topology.dim - 1, domain.topology.dim)
boundary_facets = mesh.exterior_facet_indices(domain.topology)
boundary_dofs = fem.locate_dofs_topological(
    V, domain.topology.dim - 1, boundary_facets
)
bcs = [fem.dirichletbc(g, boundary_dofs)]

# =============================================================================
# VARIATIONAL FORMULATION
# =============================================================================

# Backward Euler time discretization
F = (
    ufl.inner(T - T_n, v) * ufl.dx
    + dt * c * ufl.inner(ufl.grad(T), ufl.grad(v)) * ufl.dx
    - dt * ufl.inner(f, v) * ufl.dx
)

a, L = ufl.system(F)
a = fem.form(a)
L = fem.form(L)

# =============================================================================
# LINEAR ALGEBRA SETUP
# =============================================================================

# Assemble matrix
A = petsc.assemble_matrix(a, bcs=bcs)
A.assemble()

# Create solver
solver = PETSc.KSP().create(domain.comm)
solver.setOperators(A)
solver.setType(PETSc.KSP.Type.PREONLY)
solver.getPC().setType(PETSc.PC.Type.LU)

# Solution function
Th = fem.Function(V)

# =============================================================================
# OUTPUT FILE SETUP
# =============================================================================

# Create XDMF file for ParaView visualization
with XDMFFile(MPI.COMM_WORLD, "heat_transfer.xdmf", "w") as xdmf:
    xdmf.write_mesh(domain)

    # Write diffusion coefficient
    c.name = "diffusivity"
    xdmf.write_function(c, t)

    # Write initial condition
    Th.x.array[:] = T_n.x.array
    Th.name = "temperature"
    xdmf.write_function(Th, t)

    # =============================================================================
    # TIME STEPPING LOOP
    # =============================================================================

    step = 0
    while t < T_end:
        t += float(dt)
        step += 1

        # Assemble RHS vector
        with Th.x.petsc_vec.localForm() as loc:
            loc.set(0)

        petsc.assemble_vector(Th.x.petsc_vec, L)
        petsc.apply_lifting(Th.x.petsc_vec, [a], [bcs])
        Th.x.petsc_vec.ghostUpdate(addv=PETSc.InsertMode.ADD_VALUES,
                                   mode=PETSc.ScatterMode.REVERSE)
        petsc.set_bc(Th.x.petsc_vec, bcs)

        # Solve
        solver.solve(Th.x.petsc_vec, Th.x.petsc_vec)
        Th.x.scatter_forward()

        # Update previous solution
        T_n.x.array[:] = Th.x.array

        # Write solution
        xdmf.write_function(Th, t)

        # Print only on rank 0
        if step % 5 == 0 and rank == 0:
            print(f"Time step {step}: t = {t:.3f}")

# =============================================================================
# FINAL OUTPUT
# =============================================================================

if rank == 0:
    print("\n" + "="*60)
    print("Simulation complete!")
    print(f"Total time steps: {step}")
    print(f"Final time: {t:.3f}")
    print("Output saved to: heat_transfer.xdmf")
    print("Open with ParaView to visualize results")
    print("="*60)


Time step 5: t = 0.050
Time step 10: t = 0.100
Time step 15: t = 0.150
Time step 20: t = 0.200
Time step 25: t = 0.250
Time step 30: t = 0.300
Time step 35: t = 0.350
Time step 40: t = 0.400
Time step 45: t = 0.450
Time step 50: t = 0.500
Time step 55: t = 0.550
Time step 60: t = 0.600
Time step 65: t = 0.650
Time step 70: t = 0.700
Time step 75: t = 0.750
Time step 80: t = 0.800
Time step 85: t = 0.850
Time step 90: t = 0.900
Time step 95: t = 0.950
Time step 100: t = 1.000
Time step 105: t = 1.050
Time step 110: t = 1.100
Time step 115: t = 1.150
Time step 120: t = 1.200
Time step 125: t = 1.250
Time step 130: t = 1.300
Time step 135: t = 1.350
Time step 140: t = 1.400
Time step 145: t = 1.450
Time step 150: t = 1.500
Time step 155: t = 1.550
Time step 160: t = 1.600
Time step 165: t = 1.650
Time step 170: t = 1.700
Time step 175: t = 1.750
Time step 180: t = 1.800
Time step 185: t = 1.850
Time step 190: t = 1.900
Time step 195: t = 1.950
Time step 200: t = 2.000

Simulation complete

In [3]:
from google.colab import files
files.download('heat_transfer.xdmf')
files.download('heat_transfer.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>